In [ ]:
# pip install datasets

In [ ]:
# !pip install datasets transformers[sentencepiece] sacrebleu -q

In [ ]:
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM,DataCollatorForSeq2Seq,AdamWeightDecay

In [ ]:
model_checkpoint = "HaiderSultanArc/t5-small-english-to-urdu"

**model** : HaiderSultanArc/t5-small-english-to-urdu

source : https://huggingface.co/HaiderSultanArc/t5-small-english-to-urdu

**Dataset** : https://huggingface.co/datasets/HaiderSultanArc/MT-Urdu-English

In [ ]:
# loading online dataset from hugging face
raw_dataset = load_dataset("HaiderSultanArc/MT-Urdu-English")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/603 [00:00<?, ?B/s]

(…)-00000-of-00003-4380223537567648.parquet:   0%|          | 0.00/233M [00:00<?, ?B/s]

(…)-00001-of-00003-4b574fda4b8ddfe8.parquet:   0%|          | 0.00/233M [00:00<?, ?B/s]

(…)-00002-of-00003-1413ab686eef168c.parquet:   0%|          | 0.00/233M [00:00<?, ?B/s]

(…)-00000-of-00001-8d5468f2cad985ff.parquet:   0%|          | 0.00/175M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5646138 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1411535 [00:00<?, ? examples/s]

In [ ]:
# viewing dataset format
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'ur'],
        num_rows: 5646138
    })
    test: Dataset({
        features: ['en', 'ur'],
        num_rows: 1411535
    })
})

In [ ]:
# displaying some samples
raw_dataset["train"][0:5]

{'en': ['So, Are You In The Market?',
  'And we never asked help or even have a quiet moment with our Lord.',
  "'Then prayed I to the God of heaven'.",
  'Are you able to click on the [Create another] button at the bottom?',
  'Then another holiday is under attack.'],
 'ur': ['"تم یہاں بازار میں ؟"',
  'نہیں مانگتے مدد ہم ان سے یا رب',
  'اور میں آسمان کے خدا سے دعا کروں گا.',
  'کیا تم ایک ایک ٹیلے پر بطور کھیل تماشہ یادگار (عمارت) بنا رہے ہو ۔',
  'ایک اور فدائی حملہ ہوتا ہے.']}

**Preprocessing the data**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [15997, 108, 251, 122, 103, 126, 7096, 182, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[15997, 108, 251, 122, 103, 126, 7096, 182, 1], [547, 122, 787, 7096, 133, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["تم یہاں بازار میں ؟"]))

{'input_ids': [[128, 476, 8778, 113, 103, 630, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:

from datasets import load_dataset, DatasetDict # Import DatasetDict

reduced_train_dataset = raw_dataset['train'].select(range(1000))  # Select first 500,000 rows for training
reduced_test_dataset = raw_dataset['test'].select(range(100))  # Select first 10,000 rows for testing

# Create a new DatasetDict with the reduced datasets
reduced_dataset = DatasetDict({ # Now using DatasetDict
    'train': reduced_train_dataset,
    'test': reduced_test_dataset
})

In [ ]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "ur"

def preprocess_function(examples):
    # Accessing elements using column names instead of string indices
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # setup the tokenizer for target
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(reduced_dataset["train"][:2])

{'input_ids': [[3268, 1974, 362, 648, 291, 28103, 495, 590, 273, 1], [147, 246, 535, 1578, 731, 236, 613, 154, 103, 126, 17356, 6223, 161, 374, 719, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[5735, 476, 8778, 113, 103, 3392, 1], [145, 2455, 492, 135, 120, 111, 103, 151, 138, 218, 1]]}

In [ ]:
tokenized_dataset = reduced_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 2

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [ ]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset, epochs=num_train_epochs)

Epoch 1/2
62/62 [==============================] - 653s 10s/step - loss: 4.4310
Epoch 2/2
62/62 [==============================] - 620s 10s/step - loss: 4.3096


In [ ]:
model.save_pretrained("english-to-urdu-model")

**Model Testing**

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

model_save_path = '/content/drive/My Drive/english-to-urdu-model'

# model.save_pretrained(model_save_path)

# ... (Later, when you want to load the model) ...

# Load the model using from_pretrained
loaded_model = TFAutoModelForSeq2SeqLM.from_pretrained(model_save_path)

print("Model loaded successfully!")

Mounted at /content/drive


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/drive/My Drive/english-to-urdu-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Model loaded successfully!


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# model = TFAutoModelForSeq2SeqLM.from_pretrained("english-to-urdu-model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

In [ ]:
input_text = "hey! how are you."

tokenized = tokenizer([input_text], return_tensors='np')
out = loaded_model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor([[   0  117  238  117  238  286  111 7512  727  207    1]], shape=(1, 11), dtype=int32)


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))